## Step1：全局定义

In [1]:
# 各种 import
from transformers import pipeline, AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer
from datasets import load_dataset
import numpy as np
import evaluate
import os

C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# 全局变量
os.environ['HF_HOME'] = 'E:\PycharmProjects\huggingface'
os.environ['HF_HUB_CACHE'] = 'E:\PycharmProjects\huggingface\cache'
os.environ['HF_ENDPOINT'] = 'https://hf-mirror.com'

model_raw_dir = "models/bert-base-cased"
model_trained_dir = "models/bert-base-cased-finetune-yelp"
dataset_dir = "datasets/yelp_review_full"
batch_size = 10

tokenizer = AutoTokenizer.from_pretrained(model_raw_dir)
model = AutoModelForSequenceClassification.from_pretrained(model_raw_dir, num_labels=5)
dataset = load_dataset(dataset_dir)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at models/bert-base-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [3]:
# 数据集预处理
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

# train_set = tokenized_datasets["train"].shuffle(seed=42).select(range(100))
# test_set = tokenized_datasets["test"].shuffle(seed=42).select(range(100))

# 65w条训练数据5w条测试数据
train_set = tokenized_datasets["train"]
test_set = tokenized_datasets["test"]

## Step2：开始训练

In [4]:
# 加载 evaluate
metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

In [5]:
# 加载训练参数和训练器
# 1个epoch，batchsize=10，需要6.5w steps，1.3w steps评价一次，一共评价5次
training_args = TrainingArguments(output_dir=model_trained_dir,
                                  evaluation_strategy="steps", 
                                  per_device_train_batch_size=batch_size,
                                  num_train_epochs=1,
                                  # logging_strategy="epoch",
                                  logging_steps=13000,
                                  save_strategy="epoch",
                                  overwrite_output_dir = True)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_set,
    eval_dataset=test_set,
    compute_metrics=compute_metrics,
)

C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\transformers\training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [6]:
trainer.train()
trainer.save_model(model_trained_dir)
trainer.save_state()

C:\Users\wangx\miniconda3\envs\peft\lib\site-packages\transformers\models\bert\modeling_bert.py:435: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\cb\pytorch_1000000000000\work\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:455.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


Step,Training Loss,Validation Loss,Accuracy
13000,0.939500,0.872450,0.625620
26000,0.862400,0.831018,0.643120
39000,0.815200,0.773721,0.662100
52000,0.781800,0.757856,0.669680
65000,0.748000,0.736981,0.679300
